In [1]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [2]:
doc=nlp(u'9 99 9999 ghjkl')

In [3]:
doc[0].is_alpha

False

In [4]:
doc[3].is_alpha

True

In [5]:
doc=nlp(u'"We\'re moving to L.A.!"')
print(doc)

"We're moving to L.A.!"


In [6]:
for t in doc:
    print(t.text,end=' | ')

" | We | 're | moving | to | L.A. | ! | " | 

* First split happens on whitespace.
* Prefix Exception - We're split into We & 're
* Suffix Exception - L.A.! split into  L.A. and !

### NER

In [7]:
doc2= nlp(u'Apple to build a Hongkong factory for $6 million')

In [8]:
for t in doc2:
    print(t.text,end=' | ')


print()
print()
for ent in doc2.ents:
    print(ent.text,' >> ',ent.label_, ' >> ', str(spacy.explain(ent.label_)))

Apple | to | build | a | Hongkong | factory | for | $ | 6 | million | 

Apple  >>  ORG  >>  Companies, agencies, institutions, etc.
Hongkong  >>  GPE  >>  Countries, cities, states
$6 million  >>  MONEY  >>  Monetary values, including unit


In [9]:
from spacy import displacy

displacy.render(doc2,style='ent',jupyter=True)

In [10]:
displacy.render(doc2,style='dep',jupyter=True,options={'distance':90})

In [ ]:
displacy.serve(doc2,style='dep')

C:\ProgramData\Anaconda3\lib\runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



# semantic similarity

In [ ]:
doc1 = nlp('cat')

In [ ]:
doc2 = nlp('lion')

In [ ]:
doc1.similarity(doc2)

In [ ]:
ex1 = nlp('wolf dog cat fish bird')

In [ ]:
for token1 in ex1:
    for token2 in ex1:
        print((token1.text,token2.text),'similarity =>',token1.similarity(token2))

In [ ]:
mylist = [(token1.text,token2.text,token1.similarity(token2)) for token1 in ex1 for token2 in ex1]

In [ ]:
mylist

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(mylist)

In [ ]:
df.head()

In [ ]:
df.columns = ['token1','token2','similarity']

In [ ]:
df

# stop words

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
print(STOP_WORDS)

In [ ]:
nlp.vocab['the'].is_stop

In [ ]:
mysentense = nlp('This is a sentense about how to use stopwords in natural language processing')

In [ ]:
for words in mysentense:
    if words.is_stop == True:
        print(words)

In [ ]:
for words in mysentense:
    if words.is_stop == False:
        print(words)

In [ ]:
[words for words in mysentense if words.is_stop == False]

#### adding stop words

In [ ]:
STOP_WORDS.add('lol')

In [ ]:
nlp.vocab['lol']

# Noun Chunks
  
  * noun + word describing the noun
  * noun phrases
  * adnomial
  * root.text


In [ ]:
vl = nlp('The man reading the news is very tall')

In [ ]:
for token in vl.noun_chunks:
    print(token.text)

In [ ]:
for token in vl.noun_chunks:
    print(token.root.text)

In [ ]:
for token in vl.noun_chunks:
    print(token.root.text,'connecyer_text =>',token.root.head.text)

## Sentence Segmentation or Boundary Detection
* ##### Deciding where sentences begin and end
===================================================
* a) If it's a period, it ends a sentence.
* (b) If the preceding token is in the hand-compiled list of abbreviations, then it doesn't end a sentence.
* (c) If the next token is capitalized, then it ends a sentence.
* ===================================================
* Default = Uses the Dependency parser
* Custom Rule Based or Manual
* You set boundaries before parsing the doc

In [1]:
# Manual or Custom Based
def mycustom_boundary(docx):
    for token in docx[:-1]:
        if token.text == '...':
            docx[token.i+1].is_sent_start = True
    return docx

In [4]:
nlp.add_pipe(mycustom_boundary,before='parser')

In [5]:
mydoc = nlp(u"This is my first sentence...the last comment was so cuul... what if...? this is the last sentence")

In [6]:
for sentence in mydoc.sents:
    print(sentence.text)

This is my first sentence...
the last comment was so cuul...
what if...
?
this is the last sentence


## Custom rule based

In [7]:
from spacy.lang.en import English
from spacy.pipeline import SentenceSegmenter

In [10]:
def split_on_newlines(doc):
    start = 0
    seen_newline = False
    for word in doc:
        if seen_newline and not word.is_space:
            yield doc[start:word.i]
            start = word.i
            seen_newline = False
        elif word.text == '\t':
            seen_newline = True
    if start < len(doc):
        yield doc[start:len(doc)]

In [11]:
nlp = English()  # just the language with no model
sbd = SentenceSegmenter(nlp.vocab, strategy=split_on_newlines)
nlp.add_pipe(sbd)

In [12]:
doc = nlp(u"This is a great sentence\n\nThis is another comment\nAnd more")
for sent in doc.sents:
    print(sent.text)

This is a great sentence

This is another comment
And more


# Intent Classification with Rasa NLU and SpaCy
* A Libary for intent recognition and entity extraction based on SpaCy and Sklearn
* NLP = NLU+NLG+ More
* NLP = understand,process,interprete everyday human language
* NLU = unstructured inputs and convert them into a structured form that a machine can understand and act upon
* Uses
* Chatbot task
* NL understanding
* Intent classification

###### Installation
* pip install rasa_nlu
* python -m rasa_nlu.server &
* sklearn_crfsuite
###### using spacy as backend
* pip install rasa_nlu[spacy]
* python -m spacy download en_core_web_md
* python -m spacy link en_core_web_md en

= = Dataset = =

* demo-rasa.json
* config_spacy.yaml

In [4]:
pip install rasa_nlu


  Found existing installation: jsonschema 3.0.2
    Uninstalling jsonschema-3.0.2:


ERROR: spyder 3.3.6 requires pyqt5<5.13; python_version >= "3", which is not installed.
ERROR: spyder 3.3.6 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
ERROR: jupyterlab-server 1.0.6 has requirement jsonschema>=3.0.1, but you'll have jsonschema 2.6.0 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\jsonschema-3.0.2.dist-info\\COPYING'
Consider using the `--user` option or check the permissions.



Note: you may need to restart the kernel to use updated packages.


In [6]:
python -m rasa_nlu.server 

SyntaxError: invalid syntax (<ipython-input-6-da6270326668>, line 1)

In [8]:
pip install sklearn_crfsuite

  Created wheel for tabulate: filename=tabulate-0.8.6-cp37-none-any.whl size=23279 sha256=78f932b93131a13109744f63177cc3eb0b9e7517d1a7e98ef419d6704cf9f973
  Stored in directory: C:\Users\Vimannyu Singh\AppData\Local\pip\Cache\wheels\9c\9b\f4\eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
Successfully built tabulate
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install rasa_nlu[spacy]

In [ ]:
python -m spacy download en_core_web_md
python -m spacy link en_core_web_md e